In [18]:
import sys
from pathlib import Path
sys.path.append(str(Path(sys.argv[0]).absolute().parent.parent.parent.parent))
# add the entire folder to path
from src.utils import utils

In [ ]:
user = input("user: ")
password = input("password: ")
ip = input("ip: ")
key = input("key: ")

In [ ]:
from sqlalchemy import create_engine
import pandas as pd

sql_uri = f'mysql+pymysql://{user}:{password}@{ip}:3306/playerdata'

engine = create_engine(sql_uri)
sql=f'''
select 
    HEX(AES_ENCRYPT(pl.name, "{key}")) as name, 
    pl.created_at, 
    pl.updated_at, 
    pl.possible_ban, 
    pl.confirmed_ban, 
    pl.label_id, 
    lb.label,
    case 
        when label_jagex = 2 then "banned"
        else null
    end as account_status,
    phd.*
from Players pl 
join playerHiscoreDataLatest phd on pl.id = phd.Player_id
join Labels lb on pl.label_id = lb.id
where pl.label_id <> 0'''
with engine.connect() as conn:
    df = pd.read_sql(sql, params=None ,con=conn)
df

In [ ]:

df[utils.HISCORE_COLUMNS] = df[utils.HISCORE_COLUMNS].fillna(0)
df[utils.HISCORE_COLUMNS] = df[utils.HISCORE_COLUMNS].replace(-1, 0)
df["total"] = df[utils.SKILLS].sum(axis=1)

In [ ]:
df["account_status"] = df["account_status"].fillna("not banned")

In [ ]:
from datetime import datetime 
now = datetime.now().strftime("%Y-%m-%d")
now

In [ ]:
df.to_parquet(
    f'data/{now}_hiscore_data.parquet.gzip',
    compression='gzip'
)
